In [10]:
from typing import Sequence

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import multiprocess
import warnings
warnings.filterwarnings("ignore")

In [11]:
pd.set_option('display.max_columns', None)

%matplotlib inline

import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # macOS 
    data_root = "/Volumes/Brain2023/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2023/"

In [5]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache

cache_dir  = data_root

cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(cache_dir=cache_dir, use_static_cache=True)

In [31]:
import os

In [37]:
len(os.listdir(f"{data_root}/visual-behavior-neuropixels/behavior_ecephys_sessions/"))

153

In [18]:
def session_call(func: callable, session_id: int, session_type:str = None) -> tuple[int, object]:
    """
    Pull a session object from an already instantiated cache object.
    
    Parameters
    ----------
    func
        Function to apply over session. The function should take a Session object
        as it's input.
    session_id
        Session id from a cache table to pull information for.
    
    Returns
    -------
    session_id
        The session_id processed
    result
        The output of the function
    
    """
    # session = cache.get_ecephys_session(session_id)
    session = cache.get_behavior_session(session_id)
    result = func(session)
    return session_id, result

def parallel_session_map(func: callable, session_id_list: Sequence, batch_size: int = 32, cores: int = 1) -> dict:
    """Given a table from a cache, yield all the sessions as a list.
    
    This is done in batches to prevent problems with the database.
    
    Parameters
    ----------
    func
        Function to apply over all sessions
    session_id_list
        List of session_ids to grab
    batch_size
        The number of parallel queries to do.
    cores
        Number of CPUs to use.
    
    Returns
    -------
    sessions
        Dictionary of session objects from the table where keys are the session_id and values
        are the output of the function called.
    """
    max_cores = multiprocess.cpu_count()
    if cores > max_cores:
        raise AttributeError(f"Too many cpus requested. Please choose a number less than {max_cores}")
    
    batch_number = cache_table.shape[0] // batch_size
    pool = multiprocess.Pool(cores)
    batches = np.array_split(ephys_table, batch_number)
    for batch in batches:
        session_list.extend(pool.map(session_query, cache_table.index))
        
    return session_list

In [41]:
behavior_table = cache.get_behavior_session_table()
behavior_table = behavior_table.loc[~behavior_table["ecephys_session_id"].isna()]

In [43]:
ephys = cache.get_ecephys_session_table()
ephys.shape

(103, 21)

In [26]:
session_id = 1040882886
session = cache.get_behavior_session(session_id)

In [27]:
session.get_performance_metrics()
def get_hit_proportion(session):
    metrics = session.get_performance_metric()
    hit_rate = metrics["hit_trial_count"]/metrics["go_trial_count"]
    return hit_rate

In [28]:
get_hit_proportion(session)

AttributeError: 'BehaviorSession' object has no attribute 'get_performance_metric'